____

# Ho Thi Hai Yen

## Python Intermediate Programming Assignment 2

___

# 1. Import required packages and set working directory

In [180]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import os

# 2. Data cleaning & transformation

- MTR routes, fares and barrier-free facilities: https://data.gov.hk/en-data/dataset/mtr-data-routes-fares-barrier-free-facilities
- Cencus data on Hong Kong population by age and district location: 
- Geospatial data on bus stops and MTR station exits


## 2.1. MTR routes, fares and barrier-free facilities

**Add notes about what the datasets are**

In [181]:
barrier_free_facility = pd.read_csv('data/raw/barrier_free_facilities.csv')
barrier_free_facility_category = pd.read_csv('data/raw/barrier_free_facility_category.csv')
mtr_lines_and_stations = pd.read_csv('data/raw/mtr_lines_and_stations.csv')

### Barrier free facility 

In [182]:
barrier_free_facility.head()

,Station_No,Key,Value,AJTextEn,AJTextZh,Exit_Coordinate_X_Y
0,117,AJ1,N,NaN,NaN,NaN
1,117,AJ2,Y,Exit B2,B2 出口,NaN
2,117,AJ3,Y,Exit A,A 出口,NaN
3,117,AJ4,N,NaN,NaN,NaN
4,117,AJ5,N,NaN,NaN,NaN


In [183]:
barrier_free_facility = barrier_free_facility.rename(columns={'Key': 'barrier_free_facility_id',
                                                              'Value' : 'is_available',
                                                              'AJTextEn' : 'exit_location_en',
                                                              'AJTextZh' : 'exit_location_ch'})

In [184]:
barrier_free_facility.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3564 entries, 0 to 3563
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Station_No                3564 non-null   int64  
 1   barrier_free_facility_id  3564 non-null   object 
 2   is_available              3564 non-null   object 
 3   exit_location_en          165 non-null    object 
 4   exit_location_ch          165 non-null    object 
 5   Exit_Coordinate_X_Y       0 non-null      float64
dtypes: float64(1), int64(1), object(4)
memory usage: 167.2+ KB


In [185]:
barrier_free_facility.drop(columns=['Exit_Coordinate_X_Y'], inplace = True)

In [186]:
# Check for missing values in locations of facilities in case facitilities are available at the station

# 90% of the data is missing for the exit location --> drop the column
barrier_free_facility[(barrier_free_facility['is_available'] == 'Y') & (barrier_free_facility['exit_location_en'].isnull())]

,Station_No,barrier_free_facility_id,is_available,exit_location_en,exit_location_ch
7,117,AJ8,Y,NaN,NaN
9,117,HIn1,Y,NaN,NaN
10,117,HJ1,Y,NaN,NaN
11,117,HJ2,Y,NaN,NaN
13,117,HJ4,Y,NaN,NaN
...,...,...,...,...,...
3559,94,VJ4,Y,NaN,NaN
3560,94,VJ5,Y,NaN,NaN
3561,94,VJ6,Y,NaN,NaN
3562,94,VJ7,Y,NaN,NaN


In [187]:
barrier_free_facility.drop(columns=['exit_location_en', 'exit_location_ch'], inplace = True)

### Barrier free facility category

In [188]:
barrier_free_facility_category

,Item_Code,Category_Id,Category_En,Category_Zh,Facility_En,Facility_Zh,Sorting_Order
0,VJ1,VJ,Facilities for Visually Impaired,視覺受損人士設施,Tactile Guide Paths,失明人士引導徑,1
1,VJ2,VJ,Facilities for Visually Impaired,視覺受損人士設施,Escalator Audible Warning Signals,扶手電梯發聲提示器,2
2,VJ3,VJ,Facilities for Visually Impaired,視覺受損人士設施,Platform Tactile Yellow Lines,月台黃色凸條,3
3,VJ4,VJ,Facilities for Visually Impaired,視覺受損人士設施,Audible Device,發聲器,4
4,VJ5,VJ,Facilities for Visually Impaired,視覺受損人士設施,Braille,點字板,5
5,VJ6,VJ,Facilities for Visually Impaired,視覺受損人士設施,Colour Contrast Grab Poles,色彩分明的扶手桿,6
6,VJ7,VJ,Facilities for Visually Impaired,視覺受損人士設施,Buzzer,蜂鳴聲,7
7,VJ8,VJ,Facilities for Visually Impaired,視覺受損人士設施,Colour Contrast Inter-Car Barriers,色彩鮮明的欄障,8
8,VIn1,VJ,Facilities for Visually Impaired,視覺受損人士設施,Tactile Station Layout Maps,觸覺車站佈置圖,9
9,VIn2,VJ,Facilities for Visually Impaired,視覺受損人士設施,Modification of Obstructions,改裝障礙物,10


In [189]:
barrier_free_facility_category.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Item_Code      36 non-null     object
 1   Category_Id    36 non-null     object
 2   Category_En    36 non-null     object
 3   Category_Zh    36 non-null     object
 4   Facility_En    36 non-null     object
 5   Facility_Zh    36 non-null     object
 6   Sorting_Order  36 non-null     int64 
dtypes: int64(1), object(6)
memory usage: 2.1+ KB


In [190]:
# Rename Item_Code to Facility_Id for consistency
barrier_free_facility_category.rename(columns={'Item_Code': 'Facility_Id'}, inplace=True)

### MTR lines and stations

In [191]:
mtr_lines_and_stations.head()

,Line Code,Direction,Station Code,Station ID,Chinese Name,English Name,Sequence
0,AEL,DT,AWE,56.0,博覽館,AsiaWorld-Expo,1.0
1,AEL,DT,AIR,47.0,機場,Airport,2.0
2,AEL,DT,TSY,46.0,青衣,Tsing Yi,3.0
3,AEL,DT,KOW,45.0,九龍,Kowloon,4.0
4,AEL,DT,HOK,44.0,香港,Hong Kong,5.0


In [192]:
mtr_lines_and_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273 entries, 0 to 272
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Line Code     270 non-null    object 
 1   Direction     270 non-null    object 
 2   Station Code  270 non-null    object 
 3   Station ID    270 non-null    float64
 4   Chinese Name  270 non-null    object 
 5   English Name  270 non-null    object 
 6   Sequence      270 non-null    float64
dtypes: float64(2), object(5)
memory usage: 15.1+ KB


In [193]:
mtr_lines_and_stations.dropna(axis=0, how='all', inplace=True)

In [194]:
mtr_stations = mtr_lines_and_stations[['Station ID', 'Station Code', 'English Name']].drop_duplicates(keep='first')
mtr_stations

,Station ID,Station Code,English Name
0,56.0,AWE,AsiaWorld-Expo
1,47.0,AIR,Airport
2,46.0,TSY,Tsing Yi
3,45.0,KOW,Kowloon
4,44.0,HOK,Hong Kong
...,...,...,...
241,3.0,TST,Tsim Sha Tsui
261,86.0,OCP,Ocean Park
262,87.0,WCH,Wong Chuk Hang
263,88.0,LET,Lei Tung


### Merge tables

In [195]:
barrier_free_facility_details = barrier_free_facility \
    .merge(barrier_free_facility_category, how='left', left_on='barrier_free_facility_id', right_on='Facility_Id') \
    .merge(mtr_stations, how='left', left_on='Station_No', right_on='Station ID')
    
#Drop unnecessary columns
barrier_free_facility_details.drop(columns=['Facility_Id', 'Category_Zh', 'Facility_Zh', 'Sorting_Order', 'Station ID'],
                                   inplace=True)

In [196]:
barrier_free_facility_details.rename(columns={'barrier_free_facility_id': 'facility_id',
                                              'Station_No': 'station_no',
                                              'Category_Id': 'facility_category_id',
                                              'Category_En': 'facility_category_en',
                                              'Facility_En': 'facility_en',
                                              'Station Code': 'station_code',
                                              'English Name': 'station_name_en'}, 
                                     inplace=True)

In [197]:
barrier_free_facility_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3564 entries, 0 to 3563
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   station_no            3564 non-null   int64 
 1   facility_id           3564 non-null   object
 2   is_available          3564 non-null   object
 3   facility_category_id  3564 non-null   object
 4   facility_category_en  3564 non-null   object
 5   facility_en           3564 non-null   object
 6   station_code          3492 non-null   object
 7   station_name_en       3492 non-null   object
dtypes: int64(1), object(7)
memory usage: 222.9+ KB


In [198]:
# Check rows with missing values in station name and station code
barrier_free_facility_details[barrier_free_facility_details.isna().any(axis=1)]

,station_no,facility_id,is_available,facility_category_id,facility_category_en,facility_en,station_code,station_name_en
2196,70,AJ1,N,AJ,System Accessibility,Same Level,NaN,NaN
2197,70,AJ2,N,AJ,System Accessibility,Ramp,NaN,NaN
2198,70,AJ3,N,AJ,System Accessibility,Lift - Unpaid Area,NaN,NaN
2199,70,AJ4,N,AJ,System Accessibility,Lift - Unpaid Area (Non MTR managed facility),NaN,NaN
2200,70,AJ5,Y,AJ,System Accessibility,Stair Lift,NaN,NaN
...,...,...,...,...,...,...,...,...
3379,888,VJ4,N,VJ,Facilities for Visually Impaired,Audible Device,NaN,NaN
3380,888,VJ5,N,VJ,Facilities for Visually Impaired,Braille,NaN,NaN
3381,888,VJ6,N,VJ,Facilities for Visually Impaired,Colour Contrast Grab Poles,NaN,NaN
3382,888,VJ7,N,VJ,Facilities for Visually Impaired,Buzzer,NaN,NaN


In [199]:
# Convert yes/no values to boolean
barrier_free_facility_details['is_available'].replace({'Y': True, 'N': False}, inplace=True)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_29956\1047034746.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  barrier_free_facility_details['is_available'].replace({'Y': True, 'N': False}, inplace=True)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_29956\1047034746.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_down

In [200]:
barrier_free_facility_details

,station_no,facility_id,is_available,facility_category_id,facility_category_en,facility_en,station_code,station_name_en
0,117,AJ1,False,AJ,System Accessibility,Same Level,LOP,Long Ping
1,117,AJ2,True,AJ,System Accessibility,Ramp,LOP,Long Ping
2,117,AJ3,True,AJ,System Accessibility,Lift - Unpaid Area,LOP,Long Ping
3,117,AJ4,False,AJ,System Accessibility,Lift - Unpaid Area (Non MTR managed facility),LOP,Long Ping
4,117,AJ5,False,AJ,System Accessibility,Stair Lift,LOP,Long Ping
...,...,...,...,...,...,...,...,...
3559,94,VJ4,True,VJ,Facilities for Visually Impaired,Audible Device,EXC,Exhibition Centre
3560,94,VJ5,True,VJ,Facilities for Visually Impaired,Braille,EXC,Exhibition Centre
3561,94,VJ6,True,VJ,Facilities for Visually Impaired,Colour Contrast Grab Poles,EXC,Exhibition Centre
3562,94,VJ7,True,VJ,Facilities for Visually Impaired,Buzzer,EXC,Exhibition Centre


## 2.2. Population by age and district

In [201]:
census_url = "https://idds.census2021.gov.hk/api/query?cv.AGE=0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85-10000&cv.AREA=11,12,13,14,27,23,24,25,26,31,32,33,34,35,36,37,38,39&sv.RP=RP_NPER&period=2021&lang=en"

response = requests.get(census_url)
census_data = response.json()

In [202]:
census_data['dataSet']

[{'year': 2021,
  'svCode': 'RP',
  'svDesc': 'Population',
  'spCode': 'RP_NPER',
  'spDesc': 'Number of Persons',
  'ccList': [{'ccCode': '11',
    'ccDesc': 'Central and Western',
    'cvCode': 'AREA',
    'cvDesc': 'Area - for example : District Council District, New Town'},
   {'ccCode': '0-4', 'ccDesc': '0 - 4', 'cvCode': 'AGE', 'cvDesc': 'Age'}],
  'unit': 'Persons',
  'figure': '7583'},
 {'year': 2021,
  'svCode': 'RP',
  'svDesc': 'Population',
  'spCode': 'RP_NPER',
  'spDesc': 'Number of Persons',
  'ccList': [{'ccCode': '11',
    'ccDesc': 'Central and Western',
    'cvCode': 'AREA',
    'cvDesc': 'Area - for example : District Council District, New Town'},
   {'ccCode': '10-14', 'ccDesc': '10 - 14', 'cvCode': 'AGE', 'cvDesc': 'Age'}],
  'unit': 'Persons',
  'figure': '8192'},
 {'year': 2021,
  'svCode': 'RP',
  'svDesc': 'Population',
  'spCode': 'RP_NPER',
  'spDesc': 'Number of Persons',
  'ccList': [{'ccCode': '11',
    'ccDesc': 'Central and Western',
    'cvCode': 'AR

In [217]:
census_df = pd.json_normalize(census_data['dataSet'])
census_df['id'] = range(1, len(census_df) + 1)
census_df

,year,svCode,svDesc,spCode,spDesc,ccList,unit,figure,id
0,2021,RP,Population,RP_NPER,Number of Persons,"[{'ccCode': '11', 'ccDesc': 'Central and Weste...",Persons,7583,1
1,2021,RP,Population,RP_NPER,Number of Persons,"[{'ccCode': '11', 'ccDesc': 'Central and Weste...",Persons,8192,2
2,2021,RP,Population,RP_NPER,Number of Persons,"[{'ccCode': '11', 'ccDesc': 'Central and Weste...",Persons,6989,3
3,2021,RP,Population,RP_NPER,Number of Persons,"[{'ccCode': '11', 'ccDesc': 'Central and Weste...",Persons,9790,4
4,2021,RP,Population,RP_NPER,Number of Persons,"[{'ccCode': '11', 'ccDesc': 'Central and Weste...",Persons,15723,5
...,...,...,...,...,...,...,...,...,...
319,2021,RP,Population,RP_NPER,Number of Persons,"[{'ccCode': '39', 'ccDesc': 'Islands', 'cvCode...",Persons,10149,320
320,2021,RP,Population,RP_NPER,Number of Persons,"[{'ccCode': '39', 'ccDesc': 'Islands', 'cvCode...",Persons,7376,321
321,2021,RP,Population,RP_NPER,Number of Persons,"[{'ccCode': '39', 'ccDesc': 'Islands', 'cvCode...",Persons,3652,322
322,2021,RP,Population,RP_NPER,Number of Persons,"[{'ccCode': '39', 'ccDesc': 'Islands', 'cvCode...",Persons,2646,323


In [218]:
# Explode the list into separate rows
explored_ccList = census_df.explode('ccList')

# Expand dictionaries into columns
final_ccList = pd.concat(
    [explored_ccList.drop('ccList', axis=1), explored_ccList['ccList'].apply(pd.Series)],
    axis=1
)

final_ccList.drop(columns=['ccCode', 'cvCode', 'cvDesc'], inplace=True)
final_ccList

,year,svCode,svDesc,spCode,spDesc,unit,figure,id,ccDesc
0,2021,RP,Population,RP_NPER,Number of Persons,Persons,7583,1,Central and Western
0,2021,RP,Population,RP_NPER,Number of Persons,Persons,7583,1,0 - 4
1,2021,RP,Population,RP_NPER,Number of Persons,Persons,8192,2,Central and Western
1,2021,RP,Population,RP_NPER,Number of Persons,Persons,8192,2,10 - 14
2,2021,RP,Population,RP_NPER,Number of Persons,Persons,6989,3,Central and Western
...,...,...,...,...,...,...,...,...,...
321,2021,RP,Population,RP_NPER,Number of Persons,Persons,3652,322,75 - 79
322,2021,RP,Population,RP_NPER,Number of Persons,Persons,2646,323,Islands
322,2021,RP,Population,RP_NPER,Number of Persons,Persons,2646,323,80 - 84
323,2021,RP,Population,RP_NPER,Number of Persons,Persons,3473,324,Islands


In [219]:
final_ccList_area = final_ccList.iloc[range(0, len(final_ccList), 2)]

In [220]:
final_ccList_age = final_ccList.iloc[range(1, len(final_df), 2)]

In [221]:
census_df = census_df.merge(final_ccList_area, how='left', on='id') \
    .merge(final_ccList_age, how='left', on='id')

In [222]:
census_df

,year_x,svCode_x,svDesc_x,spCode_x,spDesc_x,ccList,unit_x,figure_x,id,year_y,...,figure_y,ccDesc_x,year,svCode,svDesc,spCode,spDesc,unit,figure,ccDesc_y
0,2021,RP,Population,RP_NPER,Number of Persons,"[{'ccCode': '11', 'ccDesc': 'Central and Weste...",Persons,7583,1,2021,...,7583,Central and Western,2021,RP,Population,RP_NPER,Number of Persons,Persons,7583,0 - 4
1,2021,RP,Population,RP_NPER,Number of Persons,"[{'ccCode': '11', 'ccDesc': 'Central and Weste...",Persons,8192,2,2021,...,8192,Central and Western,2021,RP,Population,RP_NPER,Number of Persons,Persons,8192,10 - 14
2,2021,RP,Population,RP_NPER,Number of Persons,"[{'ccCode': '11', 'ccDesc': 'Central and Weste...",Persons,6989,3,2021,...,6989,Central and Western,2021,RP,Population,RP_NPER,Number of Persons,Persons,6989,15 - 19
3,2021,RP,Population,RP_NPER,Number of Persons,"[{'ccCode': '11', 'ccDesc': 'Central and Weste...",Persons,9790,4,2021,...,9790,Central and Western,2021,RP,Population,RP_NPER,Number of Persons,Persons,9790,20 - 24
4,2021,RP,Population,RP_NPER,Number of Persons,"[{'ccCode': '11', 'ccDesc': 'Central and Weste...",Persons,15723,5,2021,...,15723,Central and Western,2021,RP,Population,RP_NPER,Number of Persons,Persons,15723,25 - 29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,2021,RP,Population,RP_NPER,Number of Persons,"[{'ccCode': '39', 'ccDesc': 'Islands', 'cvCode...",Persons,10149,320,2021,...,10149,Islands,2021,RP,Population,RP_NPER,Number of Persons,Persons,10149,65 - 69
320,2021,RP,Population,RP_NPER,Number of Persons,"[{'ccCode': '39', 'ccDesc': 'Islands', 'cvCode...",Persons,7376,321,2021,...,7376,Islands,2021,RP,Population,RP_NPER,Number of Persons,Persons,7376,70 - 74
321,2021,RP,Population,RP_NPER,Number of Persons,"[{'ccCode': '39', 'ccDesc': 'Islands', 'cvCode...",Persons,3652,322,2021,...,3652,Islands,2021,RP,Population,RP_NPER,Number of Persons,Persons,3652,75 - 79
322,2021,RP,Population,RP_NPER,Number of Persons,"[{'ccCode': '39', 'ccDesc': 'Islands', 'cvCode...",Persons,2646,323,2021,...,2646,Islands,2021,RP,Population,RP_NPER,Number of Persons,Persons,2646,80 - 84


In [223]:
census_df.columns

Index(['year_x', 'svCode_x', 'svDesc_x', 'spCode_x', 'spDesc_x', 'ccList',
       'unit_x', 'figure_x', 'id', 'year_y', 'svCode_y', 'svDesc_y',
       'spCode_y', 'spDesc_y', 'unit_y', 'figure_y', 'ccDesc_x', 'year',
       'svCode', 'svDesc', 'spCode', 'spDesc', 'unit', 'figure', 'ccDesc_y'],
      dtype='object')

In [224]:
population_area_age = census_df[['ccDesc_y', 'ccDesc_x', 'figure']]
population_area_age.rename(columns={'ccDesc_x': 'area', 
                          'ccDesc_y': 'age_group', 
                          'figure': 'population'}, inplace=True)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_29956\1192056181.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population_area_age.rename(columns={'ccDesc_x': 'area',


In [ ]:
population_area_age

,age_group,area,population
0,0 - 4,Central and Western,7583
1,10 - 14,Central and Western,8192
2,15 - 19,Central and Western,6989
3,20 - 24,Central and Western,9790
4,25 - 29,Central and Western,15723
...,...,...,...
319,65 - 69,Islands,10149
320,70 - 74,Islands,7376
321,75 - 79,Islands,3652
322,80 - 84,Islands,2646


In [ ]:
# Total population by area
population_area_age['population'] = population_area_age['population'].astype(int)
population_by_area = population_area_age.groupby('area')['population'].sum().reset_index()

population_by_area

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_29956\2034622670.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population_area_age['population'] = population_area_age['population'].astype(int)


In [229]:
population_area_age = population_area_age.merge(population_by_area, how='left', on='area', suffixes=('', '_total'))
population_area_age

,age_group,area,population,population_total
0,0 - 4,Central and Western,7583,235953
1,10 - 14,Central and Western,8192,235953
2,15 - 19,Central and Western,6989,235953
3,20 - 24,Central and Western,9790,235953
4,25 - 29,Central and Western,15723,235953
...,...,...,...,...
319,65 - 69,Islands,10149,185282
320,70 - 74,Islands,7376,185282
321,75 - 79,Islands,3652,185282
322,80 - 84,Islands,2646,185282


In [232]:
population_area_age['population_percentage'] = population_area_age['population'] / population_area_age['population_total'] * 100

population_area_age


,age_group,area,population,population_total,population_percentage
0,0 - 4,Central and Western,7583,235953,3.213776
1,10 - 14,Central and Western,8192,235953,3.471878
2,15 - 19,Central and Western,6989,235953,2.962031
3,20 - 24,Central and Western,9790,235953,4.149131
4,25 - 29,Central and Western,15723,235953,6.663615
...,...,...,...,...,...
319,65 - 69,Islands,10149,185282,5.477596
320,70 - 74,Islands,7376,185282,3.980959
321,75 - 79,Islands,3652,185282,1.971050
322,80 - 84,Islands,2646,185282,1.428093
